How to use: 

1. Download ffmpeg file and put it somewhere on your computer.
2. Define the path to the ffmpeg.exe file on your computer as "FFmpeg1 = path", e.g.
    Ffmpeg1 = "C:/Users/blablabla/ffmpeg.exe"
3. Run the c++ code and save the .txt files as follows:
    folder structure:
        dataset1
        output (this can be an empty or a filled folder)
        src (this is the folder with the git stuff)
4. If you want to have a movie of your dataset, then put x=1, if you do not want to have a movie, then put x=0. 
5. If you want a burn-in of the dataset, then put y=number of burn-in times, e.g. if you want to "discard" the first 30 time steps, then out y=30. 
6. run this code with: evaluation("dataset1",x,y), e.g. evaluation("dataset1",0,30).

In [ ]:
#define path to ffmpeg.exe file
FFmpeg1 = "C:/Users/Monique/Dropbox/MA_1/MA_2_Physics-of-Life/project/evaluation/FFmpeg/bin/ffmpeg.exe"

#import packages
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def update(i,positions,text,position_x,position_y,N,angles,time):
    #update(time, any further things which should be updated in animation)
    positions.set_offsets(np.transpose([position_x[i*N:(i+1)*N],position_y[i*N:(i+1)*N]]))
    positions.set_array(np.asarray([angles[i*N:(i+1)*N]/(2*np.pi),np.linspace(0,1,N)]).flatten())
    text.set_text("Time: {:.1f}".format(time[i*N]))
    return positions,text

def evaluation(foldername,movie,burnin_start):
    #'movie=1 .. movie on'
    #'movie=0 .. movie off'
    #change to current directory
    path = "../"+foldername+"/"
    #evaluate all .text files in folder
    #import glob
    pathdata = glob.glob(path+"*.txt")
    #output
    rho = []
    eta = []
    polarisat = []
    v_a = []
    #save calculations in .txt file
    plotpath = "../plots/"
    f = open(plotpath+'evaluation_'+foldername+'.txt', 'w')
    f.write("N   velocity   L   eta   r   rho   v_a   P   N   polar_interact_prob   burnin={}\n".format(burnin_start))
    for d in range(np.size(pathdata)):
        #extract input parameters from the txt file
        lines = np.loadtxt(pathdata[d], dtype=str, skiprows=1, max_rows=10, unpack=True)
        for i,l in enumerate(lines):
            k = l.find("=")
            value = eval(l[k+1:])
            if i==0: dim = value
            elif i==1: agent_number = value
            elif i==2: velocity = value
            elif i==3: box_size = value
            elif i==4: noise_strength = value
            elif i==5: neighborhood_radius = value
            elif i==6: pbc = value
            elif i==7: time_total = value
            elif i==8: time_step = value
            elif i==9: polar_interact_prob = value
        #rename agent_number as N
        N = agent_number
        #burnin = 13 # for all times
        #burnin = 13 + 30*N # for time 30 to 130
        burnin = 13 + burnin_start*N
        time, agent_index, position_x, position_y, angles = np.loadtxt(pathdata[d],float,skiprows=burnin,unpack=True)
        print("time:", max(time), " for eta = ", noise_strength)
        #modulos angles from (-pi,pi) to (0,2pi)
        angles = angles % (2*np.pi)
        #movie
        fig = plt.figure(figsize=(5,5))
        plt.set_cmap(cm.hsv)
        positions = plt.scatter(position_x[0:N],position_y[0:N],c=cm.hsv(angles[0:N]/(2*np.pi)),s=1)
        plt.title(r"N = {}, v = {}, L = {}, $\eta$ = {}, r = {}".format(N,velocity,box_size,noise_strength,neighborhood_radius))
        text = plt.text(0,0,"Time: 0")
        #update function for movie
        if movie:
            #generate a movie
            animation = anim.FuncAnimation(fig,update,np.size(time)//N,fargs=(positions,text,position_x,position_y,N,angles,time),interval=100,blit=True,repeat=False)
            #for MP4 files
            Writer = anim.FFMpegWriter(fps=5, bitrate=1800)
            plt.rcParams["animation.ffmpeg_path"] = FFmpeg1
            #save movie
            plt.xlim((0, box_size))
            plt.ylim((0, box_size))
            animation.save(pathdata[d]+".mp4", writer=Writer)
            plt.show()
        plt.close()
        del(fig)
        #calculations
        polarisation = []
        polarop = []
        nematicop = []
        meanangle = []
        av_v = []
        for i in range (np.size(time)//N):
            theta = angles[i*N:(i+1)*N] #in radians for cos functions
            polarisation.append((1/N) * np.abs(np.sum(theta))) #polarisation per frame
            meanangle.append(np.mean(theta))
            s = np.sqrt(np.sum(np.cos(theta))**2 + np.sum(np.sin(theta))**2)
            av_v.append(s/(N))
            polarop.append(np.abs(np.mean(np.exp(1j*np.array(theta)))))
            nematicop.append(np.abs(np.mean(np.exp(2j*np.array(theta)))))
        #figure for polar and nematic order parameter
        plt.figure()
        plt.plot(polarop,label="polar")
        plt.plot(nematicop,label="nematic")
        plt.legend()
        plt.title("N={}, v={}, L={}, eta={}\n".format(N, velocity, box_size, noise_strength))
        #plt.savefig(pathdata[d]+".png")
        plt.show()
        #frames
        xdata = list(range(1,(np.size(time)//N)+1))
        #further calculations:
        #overall particle density
        rho.append(N/(box_size)**2)
        #polarisation: P = 1/N*abs(sum(theta))
        polarisat.append(np.mean(polarisation))
        #noise_strength = eta
        eta.append(noise_strength)
        #average velocity
        v_a.append(np.mean(av_v))
        #further calculations:
        #density
        density = N/(box_size)**2
        #average velocity
        av_velocity = np.mean(av_v)
        #polar order parameter
        #P = np.abs(np.mean(np.exp(1j*np.array(meanangle))))
        Pop = np.abs(np.mean(np.exp(1j*np.array(angles))))
        #nematic order parameter
        #N = np.abs(np.mean(np.exp(2j*np.array(meanangle))))
        Nop = np.abs(np.mean(np.exp(2j*np.array(angles))))
        #save calculated parameters in .txt file
        f.write("{}  {}  {}  {}  {}  {}  {}  {}  {}  {}\n".format(N, velocity, box_size, noise_strength, neighborhood_radius, density, av_velocity, Pop, Nop, polar_interact_prob))
    f.close()
    return print("finished")

In [ ]:
#run the code
evaluation2("datasset1",0,20)